In [ ]:
import numpy as np
import sep

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.figsize'] = [10.,8.]

### open the FITS file

In [ ]:
#read image into a 2-D numpy array
fname = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)
hdu_list.info()

In [ ]:
#access image by indexing hdu_list
image_data = hdu_list[0].data

In [ ]:
#data is stored as a 2D numpy array. Show the shape of the array
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

In [ ]:
hdu_list.close()

### show the data

In [ ]:
#show the image
m,s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation='nearest',cmap='bone', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('image_1.png')

### background subtraction

In [ ]:
#measure spatially varying background on image
#change byte data to read image
image_data_new = image_data.byteswap().newbyteorder()
bkg = sep.Background(image_data_new)

In [ ]:
bkg = sep.Background(image_data_new, bw=64, bh=64, fw=3, fh=3)

In [ ]:
#get global mean and noise of image's background
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
#evaluate background as 2-D array but same size as original image
bkg_image = bkg.back()
#bkg_image = np.array(bkg)

In [ ]:
#show background
plt.imshow(bkg_image,interpolation='nearest',cmap='bone',origin='lower')
plt.colorbar();
plt.savefig('image_2.png')

In [ ]:
#evaluate background noise as 2-D array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
#show background noise
plt.imshow(bkg_rms,interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
plt.savefig('image_3.png')

In [ ]:
#subtract background
image_data_sub = image_data - bkg

### object detection

In [ ]:
#set detection threshold to be a constant value of 1.5*sigma
#sigma=global background rms
objects = sep.extract(image_data_sub, 1.5, err=bkg.globalrms)

In [ ]:
#number of objects detected
len(objects)

In [ ]:
#over-plot the object coordinates with some parameters on the image
#this will check where the detected objects are
from matplotlib.patches import Ellipse

#plot background-subtracted image
fig, ax = plt.subplots()
m,s = np.mean(image_data_sub), np.std(image_data_sub)
im = ax.imshow(image_data_sub,interpolation='nearest',cmap='gray',
               vmin=m-s,vmax=m+s,origin='lower')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i],objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i]*180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('image_4.png')

In [ ]:
#see available fields
objects.dtype.names

### aperture photometry

In [ ]:
#perform circular aperture photometry 
#with a 3 pixel radius at locations of the objects
flux, fluxerr, flag = sep.sum_circle(image_data_sub,objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
#show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### histogram their fluxes

In [ ]:
#convert to AB magnitudes
header = fits.getheader(fname)
x = header["PHOTFLAM"]
y = header["PHOTPLAM"]

#calculate zero pt 
zp_ambag = -2.5*np.log10(x) - 5*np.log10(y) - 2.408
abmags = -2.5*np.log10(flux) + zp_ambag

#plot AB magnitudes
plt.hist(abmags);